In [54]:
# Ensure project root (OpenCEP) is on sys.path for imports like `from base.Pattern import Pattern`
import sys, os, pathlib

nb_dir = (
    pathlib.Path(__file__).parent if "__file__" in globals() else pathlib.Path.cwd()
)
project_root = str((nb_dir / "..").resolve())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [55]:
from datetime import timedelta
from CEP import CEP
from base.Pattern import Pattern
from base.PatternStructure import (
    SeqOperator,
    PrimitiveEventStructure,
    KleeneClosureOperator,
)
from condition.CompositeCondition import AndCondition
from condition.BaseRelationCondition import SmallerThanCondition
from condition.Condition import Variable, SimpleCondition
from condition.KCCondition import KCIndexCondition
from stream.FileStream import FileInputStream, FileOutputStream
from plugin.citibike.CitiBike import (
    CitiBikeTripEventTypeClassifier,
    CitiBikeDataFormatter,
)
import test

In [56]:
"""citibikeConsecutiveRidesPattern = Pattern(
    SeqOperator(
        PrimitiveEventStructure("CitiBikeTrip", "a"),
        PrimitiveEventStructure("CitiBikeTrip", "b"),
    ),
    AndCondition(
        SimpleCondition(
            Variable("a", lambda x: (x["startstationid"], x["endstationid"])),
            Variable("b", lambda x: (x["startstationid"], x["endstationid"])),
            relation_op=lambda a, b: a[1] == b[0],
        )
    ),
    timedelta(minutes=5),
)"""

'citibikeConsecutiveRidesPattern = Pattern(\n    SeqOperator(\n        PrimitiveEventStructure("CitiBikeTrip", "a"),\n        PrimitiveEventStructure("CitiBikeTrip", "b"),\n    ),\n    AndCondition(\n        SimpleCondition(\n            Variable("a", lambda x: (x["startstationid"], x["endstationid"])),\n            Variable("b", lambda x: (x["startstationid"], x["endstationid"])),\n            relation_op=lambda a, b: a[1] == b[0],\n        )\n    ),\n    timedelta(minutes=5),\n)'

In [57]:
citibikeHotPathsPattern = Pattern(
    SeqOperator(
        KleeneClosureOperator(PrimitiveEventStructure("CitiBikeTrip", "a")),
        PrimitiveEventStructure("CitiBikeTrip", "b"),
    ),
    AndCondition(
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: x["bike_id"],
            relation_op=lambda a1, a2: a1 == a2,
            offset=-1,
        ),
        KCIndexCondition(
            names={"a"},
            getattr_func=lambda x: (
                int(float(x["startstationid"])),
                int(float(x["endstationid"])),
            ),
            relation_op=lambda a1, a2: a1[0] == a2[1],
            offset=-1,
        ),
        SimpleCondition(
            Variable("a", lambda x: x[-1]["bike_id"]),
            Variable("b", lambda x: x["bike_id"]),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("a", lambda x: int(float(x[-1]["endstationid"]))),
            Variable("b", lambda x: int(float(x["startstationid"]))),
            relation_op=lambda a, b: a == b,
        ),
        SimpleCondition(
            Variable("b", lambda x: int(float(x["endstationid"]))),
            relation_op=lambda end_id: str(end_id) in {"484"},
        ),
    ),
    timedelta(minutes=60),
)

In [58]:
cep = CEP([citibikeHotPathsPattern])
events = FileInputStream("../test/EventFiles/citibike-withbikeid_smaller.txt")

In [ ]:
cep.run(
    events,
    FileOutputStream("../test/demo/Matches", "output_citibike.txt"),
    CitiBikeDataFormatter(),
)